# import funcs and load hydrographics profiles ds

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate

import xarray as xr
import dask.array as da
from dask.distributed import Client

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches # for creating legend
import matplotlib.dates as mdates # converts datetime64 to datetime

import cartopy
import cartopy.crs as ccrs # for plotting
import cartopy.feature as cfeature # for map features
from cartopy.util import add_cyclic_point # for wrapping map fully - avoiding white line on 0 deg
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.dates as mdates # converts datetime64 to datetime
import matplotlib.gridspec as gridspec # to create grid-shaped combos of axes
from mpl_toolkits import mplot3d # 3d plotting tool
import cmocean # for nice oceanography colour pallettes

#import argopy
#from argopy import DataFetcher as ArgoDataFetcher # to load Argo ds directly

import os # for finding files

import gsw # for conversion functions

from tqdm.notebook import tqdm_notebook as tqdm
import glob # for downloading data
import sys # for path to functions

import seaborn as sns

sns.set(#font='Franklin Gothic Book',
        rc={
         'axes.axisbelow': False,
         'axes.edgecolor': 'Black',
         'axes.facecolor': 'w', 
                            # '#aeaeae',
         'axes.grid': False,
         'axes.labelcolor': 'k',
         'axes.spines.right': True,
         'axes.spines.top': True,
         'figure.facecolor': 'white',
         'lines.solid_capstyle': 'round',
         'patch.edgecolor': 'k',
         'patch.force_edgecolor': True,
         'text.color': 'k',
         'xtick.bottom': True,
         'xtick.color': 'k',
         'xtick.direction': 'out',
         'xtick.top': False,
         'ytick.color': 'k',
         'ytick.direction': 'out',
         'ytick.left': True,
         'ytick.right': False},
         font_scale=1)
mpl.rcParams["figure.titlesize"] = 20
mpl.rcParams["axes.titlesize"] = 20
mpl.rcParams["axes.labelsize"] = 15
mpl.rcParams["font.size"] = 12
mpl.rcParams["xtick.labelsize"] = 12
mpl.rcParams["ytick.labelsize"] = 12
mpl.rcParams["ytick.labelright"] = False

from warnings import filterwarnings as fw
fw('ignore')

# import my own funcs
import sys
sys.path.append('/home/theospira/notebooks/projects/01-WW_climatology/functions')
from plot_formatting import circular_boundary,plot_nice_box
from inspection_funcs import boxplot

import importlib
#importlib.reload(sys.modules['inspection_plot'])

path = '/home/theospira/notebooks/projects/01-WW_climatology'

# load data

In [2]:
ds = xr.open_dataset('/home/theospira/notebooks/projects/03-WW-timeseries/data/hydrographic_profiles/hydrographic_dataset.nc')
ds

<xarray.Dataset>
Dimensions:  (n_prof: 592946, pres: 196)
Coordinates:
  * n_prof   (n_prof) int64 0 1 2 3 4 5 ... 620288 620289 620290 620291 620292
  * pres     (pres) int64 10 12 14 16 18 20 22 ... 388 390 392 394 396 398 400
    lon      (n_prof) float64 ...
    lat      (n_prof) float64 ...
    time     (n_prof) datetime64[ns] ...
Data variables: (12/22)
    temp     (n_prof, pres) float64 ...
    psal     (n_prof, pres) float64 ...
    dsource  (n_prof) object ...
    mld      (n_prof) float64 ...
    asal     (n_prof, pres) float32 ...
    ctemp    (n_prof, pres) float32 ...
    ...       ...
    ww_type  (n_prof) float64 ...
    ww_n2    (n_prof) float64 ...
    thcc     (n_prof) float64 ...
    zust     (n_prof) float64 ...
    eps      (n_prof) float64 ...
    sic      (n_prof) float64 ...
Attributes:
    description:  Interpolated dataset from 2004 to 2021. Contains MLD from p...

In [6]:
for i,d in enumerate(np.unique(ds.dsource)):
    idx = np.where(ds.dsource == d)[0]
    if d=='SOCCOM':
        ds['dsource'][idx] = 1
    else:
        ds['dsource'][idx] = i+1 

ds.dsource.attrs['description'] = "'Float':1, 'CTD':2, 'Gliders':3, 'MEOP':4,"
ds['dsource'] = ds['dsource'].astype(int)

In [4]:
#ds = xr.open_dataset('/home/theospira/notebooks/projects/WW_climatology/data/hydrographic_profiles/superseded/ww_gauss_smoothed_ds-preDec23.nc')

# calculate pressure variables as depth
arr = []
lat = ds.lat.data
for v in ['ww_cp','up_bd','lw_bd']:
    arr += gsw.z_from_p(ds[v],lat),

ds['ww_cd']   = xr.DataArray(arr[0].copy()*-1)
ds['up_bd_z'] = xr.DataArray(arr[1].copy()*-1)
ds['lw_bd_z'] = xr.DataArray(arr[2].copy()*-1)
ds['thcc_z']  = (ds.lw_bd_z - ds.up_bd_z)

# calculate z across dataset
p = np.ndarray(ds.ctemp.shape)*np.nan
l = np.ndarray(ds.ctemp.shape)*np.nan

lat  = ds.lat.data
pres = ds.pres.data
for i in range(p.shape[0]):
    p[i,:] = pres
for i in range(l.shape[1]):
    l[:,i] = lat

ds['z'] = xr.DataArray(gsw.z_from_p(p,l)*-1,dims=ds.ctemp.dims)

In [7]:
tmp = ds[['ctemp','asal','sig']].where(ds.pres<ds.mlp)
ds['ml_t']   = tmp.ctemp.mean('pres')
ds['ml_s']   = tmp.asal.mean('pres')
ds['ml_sig'] = tmp.sig.mean('pres')

In [14]:
ds.to_netcdf('/home/theospira/notebooks/projects/03-WW-timeseries/data/hydrographic_profiles/hydrographic_profiles_1.nc')

# gridding funcs

In [2]:
sys.path.append('/home/theospira/notebooks/projects/03-WW-timeseries/funcs')
from gridding import *

# time series gridding 

In [8]:
ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0])

In [11]:
np.unique(ds_ww.ww_type)

array([1., 2.])

In [20]:
len(ww_vars),len(dvars_3)

(7, 7)

In [13]:
dvars_4 = [] # 4D vars
for i in tqdm(['asal', 'ctemp', 'sig','n2']):
    dvars_4 += median_gridding_ts(ds[i],),

ww_vars = ['up_bd','ww_cp','ww_ct','ww_sa','lw_bd','thcc','ww_n2']
dvars_3 = [] # 3D vars
for i in tqdm(ww_vars):
    dvars_3 += median_gridding_ts(ds_ww[i],),

ml_vars = ['ml_t','ml_s','ml_sig','mlp','mld']
dvars_ml = [] # ML vars
for i in tqdm(ml_vars):
    dvars_ml += median_gridding_ts(ds[i],),

ww_type = mode_gridding_ts(ds_ww['ww_type'],)
n_prof  = count_gridding_ts(ds['n_prof'],)
ww_prof = count_gridding_ts(ds_ww['n_prof'],)
dsource = mode_gridding_ts(ds['dsource'],)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

...

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

In [18]:
ml_vars[]

'mlp'

In [33]:
# dvars_3
for i in range(len(ww_vars)):
    print(i, ww_vars[i])

0 up_bd
1 ww_cp
2 ww_ct
3 ww_sa
4 lw_bd
5 thcc
6 ww_n2


In [49]:
x    = 1 
time = pd.date_range('2004-01-01', '2022-01-01', freq='1M')
p    = ds.pres.data
lat  = np.arange(-79,-40+x,x) - x/2
lon  = np.arange(-180,180,x) + x/2

ds_grid = xr.Dataset(
            data_vars = dict(
                            ctemp   = (['time','lon','lat','pres'], dvars_4[1].data),
                            asal    = (['time','lon','lat','pres'], dvars_4[0].data),
                            sig     = (['time','lon','lat','pres'], dvars_4[2].data),
                            n2      = (['time','lon','lat','pres'], dvars_4[3].data),
                            mld     = (['time', 'lon','lat'], dvars_ml[4].data),
                            mlp     = (['time', 'lon','lat'], dvars_ml[3].data),
                            ml_t    = (['time', 'lon','lat'], dvars_ml[0].data),
                            ml_s    = (['time', 'lon','lat'], dvars_ml[1].data),
                            ml_sig  = (['time', 'lon','lat'], dvars_ml[2].data),
                            up_bd   = (['time', 'lon','lat'], dvars_3[0].data),
                            lw_bd   = (['time', 'lon','lat'], dvars_3[4].data),
                            ww_cp   = (['time', 'lon','lat'], dvars_3[1].data),
                            ww_ct   = (['time', 'lon','lat'], dvars_3[2].data),
                            ww_sa   = (['time', 'lon','lat'], dvars_3[3].data),
                            thcc    = (['time', 'lon','lat'], dvars_3[5].data),
                            ww_n2   = (['time', 'lon','lat'], dvars_3[6].data),
                            ww_type = (['time', 'lon','lat'], ww_type.data),
                            dsource = (['time', 'lon','lat'], dsource.data),
                            n_prof  = (['time', 'lon','lat'], n_prof.data),
                            ww_prof = (['time', 'lon','lat'], ww_prof.data),                
                            ),
            coords   = dict(
                            time    = (['time'], time),
                            lon     = (['lon'], lon),
                            lat     = (['lat'], lat),
                            pres    = (['pres'], p)
                            ),
            attrs    = dict(description=str(
    f"1 degree grid of median time series along 2dbar pressure interpolated and quality controlled Argo, MEOP, SOCCOM, CTDs, and Gliders data from 2004 to 2021. Contains good QC and Wilson et al (2019) QC. Smoothed temp and psal using Gauss smoothing with stand dev=2. Gsw funcs and WW subsequently calculated. Further QC post calculation: NaN any WW vars profiles with a lower boundary deeper than 300dbar; drop any profiles with fewer than 10 datapoints. Removed vertical profiles by cross referencing temperature profile with mixed layer temperature. If there is not a significant change from the mean ML temperature, we remove the profile (a change of 0.25°C or more for at least 5% of the profile)."),
                           note="unless specified, depth-related variables are in pressure space.")
            )

ds_grid.n_prof.attrs['description']  = 'total number of profiles per grid cell.'
ds_grid.ww_prof.attrs['description'] = 'total number of profiles containing Winter Water per grid cell; note that this is of either type of WW.'
ds_grid.ww_type.attrs['description'] = 'mode of WW type per grid cell.'
ds_grid.dsource.attrs['description'] = "'Float':1, 'CTD':2, 'Gliders':3, 'MEOP':4,"
ds_grid.mlp.attrs['description'] = 'mixed layer depth computed in pressure space (mixed layer pressure) using de Boyer Montegut et al. (2004) method.'
ds_grid.mld.attrs['description'] = 'mixed layer depth computed by converting MLP to depth (m).'

In [51]:
ds_grid.to_netcdf('/home/theospira/notebooks/projects/03-WW-timeseries/data/hydrographic_profiles/gridded-timeseries-ww-recalc1.nc')

NOTE: still to be gridded: ww_sig and regrid mlp using all MLs, not just ds_ww MLs

In [8]:
mlp   = median_gridding_ts(ds['mlp'],)
sig_c = median_gridding_ts(ds_ww['sig_c'],)

  0%|          | 0/216 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

In [11]:
np.save('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-mlp.npy',mlp)
np.save('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-sig_c.npy',sig_c)

### sum of ML and SS WW per grid cell

In [ ]:
def count_ww_profs(dsvar,condn=None):
    
    t_bins = pd.date_range('2003-12-31', '2022-01-01', freq='1M')
    # create empty array
    arr = np.ndarray([t_bins.size-1,360,40])*np.nan
    
    # define lon min and max resp
    gs = 1
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = range(0,lon_max-lon_min,gs)
    
    lat_min = -80
    lat_max = -40
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = range(0,lat_max-lat_min,gs)
    
    # group by seasons
    var = dsvar.groupby_bins(dsvar.time, bins=t_bins, labels=np.arange(len(t_bins)-1))
    
    # group into lon bins
    for t,gr_t in tqdm(var):
        var1 = gr_t.groupby_bins('lon',lon,labels=lon_labels,restore_coord_dims=True)
        
        # now group into lat bins for each lon group:
        for ln,gr_ln in var1:
            var2 = gr_ln.groupby_bins('lat',lat,labels=lat_labels,restore_coord_dims=True)
            
            # now take the mode!
            for lt,gr_lt in var2:
                if condn!=None:
                    arr[t,ln,lt] = np.where(gr_lt==condn)[0].size
                else:
                    arr[t,ln,lt] = np.where(gr_lt>0)[0].size
                    
    return arr

In [ ]:
ww_prof = count_ww_profs(ds_ww['ww_type'],)
ml_prof = count_ww_profs(ds_ml['ww_type'],condn=1)
ss_prof = count_ww_profs(ds_ss['ww_type'],condn=2)

In [ ]:
np.save('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-ww_prof.npy',ww_prof)
np.save('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-ww_ml_prof.npy',ml_prof)
np.save('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-ss_prof.npy',ss_prof)

# compute array for seasonal ice zone masking:

In [3]:
sys.path.append('/home/theospira/notebooks/projects/WW-timeseries/funcs')
from plotting import *
#from computations import calc_mean_var
from import_data import *

In [6]:
def mask_data(ds, t_ref=0, b_ref=2000):
    """
    Mask data, removing on-shelf data and any data further north than the 0°C WW core temperature isotherm.
    Remove data from 2004.
    
    Parameters:
    - ds (xarray.Dataset): The dataset containing the data to be masked.
    - t_ref (float, optional): Reference temperature value for masking. Default is 0.
    - b_ref (int, optional): Reference value for data filtering. Default is 2000.

    Returns:
    - mask (xarray.DataArray): Boolean array indicating masked cells.
    """
    
    # Retain only the 'ww_ct' variable
    ds = ds[['ww_ct']]
    
    # Cut shallow data from the dataset based on a reference value.
    ds = cut_shallow_data(ds, b_ref)

    # Remove 2004 data
    ds = ds.sel(time=slice('2005-01-01', '2022-01-01'))
    
    # Extract the circumpolar mean warm water core temperature data.
    if 'ww_ct' not in ds.data_vars.keys():
        tmp = xr.open_dataset('/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/gridded-timeseries-ww.nc')['ww_ct'].mean('time')
    else:
        tmp = ds.ww_ct.mean('time')
    
    # Add adt to dataset
    if 'adt' not in ds.data_vars.keys():
        ds = add_adt_to_ds(ds)
    
    # Find the nearest point to the reference temperature.
    tmp2 = (tmp - t_ref).__abs__()
    # Select the nearest point to the reference temperature that is south of the Polar Front.
    tmp2 = tmp2.where(tmp2.lat <= (ds.adt - -0.58).__abs__().idxmin(dim='lat')).idxmin('lat').rolling({'lon': 5}, min_periods=1).mean()

    # Create the mask: True for masked cells, False for unmasked cells
    mask = ds.lat > tmp2

    return mask

In [ ]:
d_path = '/home/theospira/notebooks/projects/WW-timeseries/data/hydrographic_profiles/'
ds = xr.open_dataset(d_path+'/ww_gauss_smoothed_ds.nc')
ds_grid = xr.open_dataset(d_path+'gridded-timeseries-ww.nc')

In [7]:
msk = mask_data(ds_grid)

In [55]:
def hydropgraphic_ds_mask(ds,msk):
    
    arr = np.zeros(ds.n_prof.size)

    lon = ds.lon.data
    lat = ds.lat.data
    for i in tqdm(range(len(ds.n_prof))):
        if msk.sel(lat=lat[i],lon=lon[i],method='nearest'):
            arr[i] = 1

    return arr.astype(np.int32)

In [ ]:
arr = hydropgraphic_ds_mask(ds,msk)

In [37]:
np.save(d_path+'masked_arr.npy',arr)